# Numba, SoA to AoS

In [46]:
import numpy as np
from numba import vectorize, float32, uint32
import random

#My_Target = 'cpu'
My_Target = 'cuda'

config_params = np.zeros(shape=5, dtype=np.float32)
config_params[0] = 80.0

birth_interval_params = np.zeros(shape=(4, 3), dtype=np.float32)
birth_interval_params[0] = [1, 2, 3]
birth_interval_params[1] = [1, 2, 3]

# print(birth_interval_params)

# individuals
NumIndividuals = 10
age_array = np.random.rand(NumIndividuals).astype(np.float32) * 80.0
alive_array = np.ones(NumIndividuals, dtype=np.uint32)
sex_array = np.array([random.random() > 0.5 for _ in range(NumIndividuals)]).astype(np.uint32)


In [ ]:
import numpy as np
from numba import vectorize

my_array = np.array([[1.0, 2.0], [3.0, 4.0]])

# @vectorize
@vectorize(['uint32(float32)'], target=My_Target)
def check_age(age):    # Elementwise!
    return age < 80.0

@vectorize(['float32(float32, float32)'], target=My_Target)
def increase(age, dt):    # Elementwise!
    age += dt
    return age

# %timeit my_add_mul(age,alive)

for i in range(10):
    age_array = increase(age_array, 3.1)
    print(age_array)

alive = check_age(age_array)
print(alive)


In [ ]:
import numpy as np
from numba import vectorize

my_array = np.array([[1.0, 2.0], [3.0, 4.0]])

# @vectorize
@vectorize(['uint32(float32)'], target=My_Target)
def check_age(age):    # Elementwise!
    return age < config_params[0]

@vectorize(['float32(float32, float32)'], target=My_Target)
def increase(age, dt):    # Elementwise!
    age += dt
    return age

# %timeit my_add_mul(age,alive)

for i in range(10):
    age_array = increase(age_array, 3.1)
    print(age_array)

alive = check_age(age_array)
print(alive)

### Input and output all arrays
When not using explicit memory transfer, results returned by guvectorize are not deterministic (looks like that some random memory address is read/returned). From the documentation my understanding was that arrays are copied without explicitely calling copy commands.
 
https://numba.pydata.org/numba-doc/dev/cuda/memory.html
https://github.com/numba/numba/issues/6159

In [49]:
import numpy as np
from numba import guvectorize, uint32, void
from numba import cuda

#My_Target = 'cpu'
My_Target = 'cuda'

my_array = np.array([[1.0, 2.0], [3.0, 4.0]])

@guvectorize(['void(float32[:], uint32[:], float32, uint32[:])'], '(n),(n),()->(n)', target=My_Target)
def update_individuals(age, alive, dt, sex):
    for i in range(age.shape[0]):
        age[i] += dt
        alive[i] = age[i] < 80
        
        if sex[i] >= 1:
            sex[i] = sex[i]+1

print(sex_array)
print(age_array.shape[0])

numba.cuda.detect()
print(numba.cuda.is_available())


d_age_array = cuda.to_device(age_array)
d_alive_array = cuda.to_device(alive_array)
d_sex_array = cuda.to_device(sex_array)

for i in range(10):
    print(i, ":")
    
    update_individuals(d_age_array, d_alive_array, 3.1, d_sex_array)
    d_sex_array.copy_to_host(sex_array)
    print(age_array)
    print(sex_array)





[11  0  0  0 11 11  0  0  0  0]
10
Found 1 CUDA devices
id 0        b'GRID V100D-4Q'                              [SUPPORTED]
                      compute capability: 7.0
                           pci device id: 0
                              pci bus id: 8
Summary:
	1/1 devices are supported
True
0 :
[105.11113  43.17657 102.0685   70.39739  38.1484   85.42315 109.93925
  80.27381 110.57912  94.0987 ]
[12  0  0  0 12 12  0  0  0  0]
1 :
[105.11113  43.17657 102.0685   70.39739  38.1484   85.42315 109.93925
  80.27381 110.57912  94.0987 ]
[13  0  0  0 13 13  0  0  0  0]
2 :
[105.11113  43.17657 102.0685   70.39739  38.1484   85.42315 109.93925
  80.27381 110.57912  94.0987 ]
[14  0  0  0 14 14  0  0  0  0]
3 :
[105.11113  43.17657 102.0685   70.39739  38.1484   85.42315 109.93925
  80.27381 110.57912  94.0987 ]
[15  0  0  0 15 15  0  0  0  0]
4 :
[105.11113  43.17657 102.0685   70.39739  38.1484   85.42315 109.93925
  80.27381 110.57912  94.0987 ]
[16  0  0  0 16 16  0  0  0  0]
5 :


In [48]:
import numpy as np
from numba import guvectorize, uint32, void
from numba import cuda

My_Target = 'cpu'
#My_Target = 'cuda'

my_array = np.array([[1.0, 2.0], [3.0, 4.0]])

@guvectorize(['void(float32[:], uint32[:], float32, uint32[:])'], '(n),(n),()->(n)', target=My_Target)
def update_individuals(age, alive, dt, sex):
    for i in range(age.shape[0]):
        age[i] += dt
        alive[i] = age[i] < 80
        
        if sex[i] >= 1:
            sex[i] = sex[i]+1

print(sex_array)
print(age_array.shape[0])


for i in range(10):
    print(i, ":")
    update_individuals(age_array, alive_array, 3.1, sex_array)
    print(age_array)
    print(sex_array)





[1 0 0 0 1 1 0 0 0 0]
10
0 :
[77.21114  15.276575 74.16851  42.497406 10.248398 57.52316  82.03926
 52.373825 82.67913  66.198715]
[2 0 0 0 2 2 0 0 0 0]
1 :
[80.31114  18.376575 77.26851  45.597404 13.348398 60.623158 85.13926
 55.473824 85.77913  69.29871 ]
[3 0 0 0 3 3 0 0 0 0]
2 :
[83.41114  21.476576 80.36851  48.697403 16.448399 63.723156 88.23926
 58.573822 88.87913  72.39871 ]
[4 0 0 0 4 4 0 0 0 0]
3 :
[86.51114  24.576576 83.468506 51.7974   19.548399 66.82316  91.33926
 61.67382  91.979126 75.49871 ]
[5 0 0 0 5 5 0 0 0 0]
4 :
[89.61114  27.676577 86.568504 54.8974   22.6484   69.92316  94.439255
 64.77382  95.079124 78.59871 ]
[6 0 0 0 6 6 0 0 0 0]
5 :
[92.711136 30.776577 89.6685   57.9974   25.7484   73.023155 97.53925
 67.87382  98.17912  81.69871 ]
[7 0 0 0 7 7 0 0 0 0]
6 :
[ 95.811134  33.876575  92.7685    61.097397  28.8484    76.12315
 100.63925   70.973816 101.27912   84.798706]
[8 0 0 0 8 8 0 0 0 0]
7 :
[ 98.91113   36.976574  95.8685    64.197395  31.9484    79.2231